## Notebook of demos, tutorial

This notebook is intended to give new users an idea of how `itermae` works to accomplish basic parsing, how to debug its usage, how to run it in parallel, and how to use multiple operations to do more complex parsing and filtering.

1. Basic usage
2. Debugging
3. Parallel chopping
4. Multiple operations

### Basic usage

`itermae` expects to have a FASTQ file piped into the standard input (STDIN) of the program, likely using the `|` character. If then applies one or more operations (denoted by `-o` or `--operation`) to each read. For each read where the operation was successful, it generates an output sequence record with a sequenced defined by the `-oseq` or `--output-seq` argument and a sequence ID defined by the `-oid` or `--output-id` argument. The output format is by default a SAM file.

Here is a minimal example that does nothing interesting, just outputs the input in SAM format:

In [28]:
cat example-data/toy.fastq | itermae -o "input > ." -oid "input.id" -oseq "input"

ExampleToyReads:1:exampleFlowCell:1:10000:10000:10000	0	*	0	255	*	=	0	0	TTCACGTCCTCGAGGTCTCTTCAGTCGTAGCAGTTCGATGCGTACGCTACAGGTCGACGGTAAGAGAGGGATGTG	AAAAAEEEA/<EAEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEE	XI:0
ExampleToyReads:1:exampleFlowCell:1:10000:10001:10001	0	*	0	255	*	=	0	0	GCTTCGTCCTCGAGGTCTCTTGGCAGACACACGCTACACGTACGCTGCAGGTCGAGGGCACGCGAGAGATGTGTG	AAAAAEEEE/AEAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE	XI:0
ExampleToyReads:1:exampleFlowCell:1:10000:10002:10002	0	*	0	255	*	=	0	0	CCCGGCGTTCGGGGAAGGACGTCAATAGTCACACAGTCCTTGACGGTATAATAACCACCATCATGGCGACCATCC	AAAAAA/A<E/<EEA6/EEE//EEE/EAEEEEEAEEEEEA/EEEEE/EEEEEE6EEEEAEEEEEEEEE/EEEEEA	XI:0
ExampleToyReads:1:exampleFlowCell:1:10000:10003:10003	0	*	0	255	*	=	0	0	CTACTGTCCACGAGGTCTCTGATGCACTGCGTTCCATGTTCGTACGCTGCAGGTCGACGGAAGGAGCGCGATGTG	AAAAAEEEE/AEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE	XI:0
ExampleToyReads:1:exampleFlowCell:1:10000:10004:10004	0	*	0	255	*	=	0	0	AAATTAGGGTCA

If the operation of output sequence definition is missing, then the program fails and tells you where it has a problem. You can omit the output ID and it will use the input ID by default. Example of failing:

In [34]:
cat example-data/toy.fastq | itermae -o "input > " -oseq "input"

Wait a second, I don't understand the operations to be done! Are there any? Maybe there's small part I'm choking on? Maybe try adding steps in one at a time in an interactive context with '--limit' set, to debug easier. Exiting...


: 1

### operations

How to write an operation? Each operation has two parts. 
There's the part on the left that is what the operation is applied to, the part on the right that is the 
regular expression to apply, and between these is ` > ` (space, greater than, space).

#### inputs
For your first operation, the input has to be `input` because that's the only available sequence, the input sequence of the sequence record. If you use mutiple operations, you can feed groups "captured" from previous operations as inputs to later operations (we talk about this later in this guide).

#### regular expressions

Then there's the regular expression. These are complex, and `itermae` is designed to expose this complexity 
to the user directly so that you can do more with the tool! Here, I will try to explain a cookbook/copy-paste 
version that you can use if you're new to regexes and need to trial-and-error the result you want:

The pattern you define is matched against the read. So the pattern `.` will match one letter of anything, 
which is why I used it in the above examples (it always matches!). Putting an `ATCG` will only match when
there's exactly a `ATCG` in the read, anywhere in the read, but exactly those four letters in that sequence.
If you want to only one character of DNA, you can use `[ATCG]` to match any one of those four letters. 
*Always use uppercase for `itermae`!* If you want to include `N`, you can do that with `[ATCGN]`. 

To match two letters, you could write `[ATCGN][ATCGN]`, or more easily `[ATCGN]{2}`. 
The `{}` braces help you specify how many to match. 
You can specify one number `{3}`, or a range `{2,5}`.
So for a DNA barcode of length 18-22 bases, you may want to specify `[ATCGN]{18,22}`.

Capture groups are the whole point of this tool, but those unfortunately have a complex syntax. A group uses
notation like `(?P<barcode>[ATCGN]{18,22})`, where `(?P<` is the start, `barcode` is the name of the group, 
and the group is whatever matches to `[ATCGN]{18,22}`, until the `)`. 

To match this barcode only when it is between defined sequences, for example priming sites, you can use a more
complex pattern like `GGTCTCT(?P<centerPart>[ATCGN]{18,22})TACGCT`. This matches a sequence where:
- There is a `GGTCTCT`
- then 18 to 22 bases of either A, T, C, G, or N
- then `TACGCT`

Crucially, it then saves that center 18 to 22 bases as a group named `centerPart`.
That means you can then only output that group, or you can put it into the output ID, or you can even match
other operation patterns against _just that group_.

For example:

In [36]:
cat example-data/toy.fastq \
    | itermae -o "input > GGTCTCT(?P<centerPart>[ATCGN]{18,22})TACGCT" -oseq "centerPart"

ExampleToyReads:1:exampleFlowCell:1:10000:10000:10000	0	*	0	255	*	=	0	0	TCAGTCGTAGCAGTTCGATGCG	EEEEEEEEEEEEEEEEEEEEEA	XI:0
ExampleToyReads:1:exampleFlowCell:1:10000:10001:10001	0	*	0	255	*	=	0	0	TGGCAGACACACGCTACACG	EEEEEEEEEEEEEEEEEEEE	XI:0
ExampleToyReads:1:exampleFlowCell:1:10000:10003:10003	0	*	0	255	*	=	0	0	GATGCACTGCGTTCCATGTTCG	EEEEEEEEEEEEEEEEEEEEEE	XI:0
ExampleToyReads:1:exampleFlowCell:1:10000:10006:10006	0	*	0	255	*	=	0	0	ATTCTGAGCGGTGCCATAGTCG	EEEEEEEEEEEEEEEEEEEEEE	XI:0
ExampleToyReads:1:exampleFlowCell:1:10000:10007:10007	0	*	0	255	*	=	0	0	ATAAGTTAGACAGGTCAGCCG	EEEEEEEEEEEEEEEEEEEEE	XI:0
ExampleToyReads:1:exampleFlowCell:1:10000:10008:10008	0	*	0	255	*	=	0	0	CACACGCACGAATTTGCATACG	EEEAEEEEEEEEEEEEAEEEEA	XI:0


This tries to match the pattern on every read, and only where it matches it them writes to the output the matched 
`centerPart` that contains the group of interest. Note that we use `"input > ` and `-oseq "centerPart"`. 
Important!

In [ ]:
#### fuzzy regular expressions

"input.id+'_'+sample.seq"

"after"

So then we get to the regular expression (?P<sample>[ATCGN]{5})(?P<after>[ATCGN]{10}). 
    
This should get you going, but regular expressions are a powerful and common feature in programming and analysis _For the best understanding, I recommend you search for "python regular expression tutorial" or the like and read a few articles that talk about this_, then consult the regex package documentation for more information about those specific modifiers.

You can change the output format using
`-of` or `--output-format` and the formats SAM, FASTQ, or FASTA. You can then write that to a file by using the `>` operator in the shell.

Wait a second, I don't understand the outputs to be done! Are there any? Maybe there's small part I'm choking on? Maybe try adding steps in one at a time in an interactive context with '--limit' set, to debug easier. Exiting...


: 1

In [2]:
cat example-data/toy.fastq \
    | itermae \
        -o "input > (?P<sample>[ATCGN]{5})(?P<after>[ATCGN]{10})" \
        -oid "input.id+'_'+sample.seq" -oseq "after"

ExampleToyReads:1:exampleFlowCell:1:10000:10000:10000_TTCAC	0	*	0	255	*	=	0	0	GTCCTCGAGG	EEEA/<EAEE	XI:0
ExampleToyReads:1:exampleFlowCell:1:10000:10001:10001_GCTTC	0	*	0	255	*	=	0	0	GTCCTCGAGG	EEEE/AEAAE	XI:0
ExampleToyReads:1:exampleFlowCell:1:10000:10002:10002_CCCGG	0	*	0	255	*	=	0	0	CGTTCGGGGA	A/A<E/<EEA	XI:0
ExampleToyReads:1:exampleFlowCell:1:10000:10003:10003_CTACT	0	*	0	255	*	=	0	0	GTCCACGAGG	EEEE/AEEAE	XI:0


Note that the '\' characters are used at the end of the line to "escape" the end of the line, thus the terminal sees the above as one long string. But humans can see it on multiple lines.

Also note that the output of the `cat` command is piped using `|` into `itermae`. This is the intended use-case of `itermae`.

Below, we go into the various aspects of what's happening, break them apart, then put them back together and show several applications and tricks that you can do with the tool.

### Input

`itermae` also has the capacity to read FASTQ or FASTQZ files from disk, but also lacks any internal parallelization. This is a design choice to limit complexity and prevent memory leaks (particularly in `regex` module), and to operate modularly with respect to other tools within a shell. 

Here is the first two records of a 'toy.fastq' file I've included in the git repo for this demo, and for you to test with:

In [ ]:
head -n 8 example-data/toy.fastq

Thus, we can use a tool like `cat` (or later `zcat` or `parallel`) to pipe that into `itermae`. 

Below is demonstrated a fairly minimal use of `itermae`, where each of the input records is parsed using a regular expression. This expression is the argument `-o "input > (?P<sample>[ATCGN]{5})(?P<after>[ATCGN]{10})"`. Let's break that down.

Each `-o` (aka `--operation`) is an *operation*. There must be at least one, and there can be many. Each operation is a fuzzy regular expression that takes some *input*, matches the pattern on the right, and stores whatever *groups* are captured by the regular expression.

In the first part of the operation we see `input > `. This means that the *input* of the sequence is exactly `input`. Putting `input` means it will take the raw input FASTQ record that's being piped into the program. Every first operation should be using this (ie starting with `input > `) because otherwise there's nothing to match against! The ` > ` symbol (with spaces on left and right!) is a delimiter to distinguish what is the input (on the left of ` > `) from the regular expression (on the right of ` > `).

So then we get to the regular expression `(?P<sample>[ATCGN]{5})(?P<after>[ATCGN]{10})`. These are complex, layered, and use a lot of funny characters. `itermae` is designed to expose this complexity to the user directly, so that the user may use the tool to execute complex operations. However, this requires a digression into regular expressions. _For the best understanding, I recommend you search for "python regular expression tutorial" or the like and read a few articles that talk about this_, then consult the [`regex` package documentation](https://pypi.org/project/regex/) for more information about those specific modifiers. Below is an abridged explanation: 
    
    "input.id+'_'+sample.seq"
    
    "after"

In [ ]:

# This should output two SAM-format entries, putting the first five bases in the
# read ID and saving the sequence as the next 10 bases after those.
#
# The above command could be equivalently written with the long-form of the
# flags as --output-id and --output-seq


In [ ]:
# Demo 02
# 
# This is the same operation as the first demo, but now demonstrating options
# to debug and get verbosity.

input_fastq=$(cat <<EOF
@NB501157:100:H5J5LBGX2:1:11101:10000:10043 1:N:0:
TTCACGTCCACGAGGTCTCTTCAGTCGTAGCAGTTCGCGTACGCTACAGGTCGACGGTAAGAGAGGGATGTGATC
+
AAAAAEEEA/<EAEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEE
@NB501157:100:H5J5LBGX2:1:11101:10000:19701 1:N:0:
CTACTGTCCACGAGGTCTCTGATGCACTGCGTTCCATGTTCGTACGCTGCAGGTCGACGGAAGGAGCGCGATGTG
+
AAAAAEEEE/AEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
EOF
)

echo
echo "With level 1 verbosity"
echo "${input_fastq}" | 
    itermae -o "input > (?P<sample>[ATCGN]{5})(?P<after>[ATCGN]{10})" \
        -oid "input.id+'_'+sample.seq" -oseq "after" \
        -v

echo
echo "With level 2 verbosity"
echo "${input_fastq}" | 
    itermae -o "input > (?P<sample>[ATCGN]{5})(?P<after>[ATCGN]{10})" \
        -oid "input.id+'_'+sample.seq" -oseq "after" \
        -v --verbose

echo
echo "With level 3 verbosity"
echo "${input_fastq}" | 
    itermae -o "input > (?P<sample>[ATCGN]{5})(?P<after>[ATCGN]{10})" \
        -oid "input.id+'_'+sample.seq" -oseq "after" \
        -v -v -v

# This should spit out a lot of outputs to standard error, that tell you exactly
# what is going on.


In [ ]:
# Demo 02
# 
# Here, we use the same FASTQ input as the previous demo, but we use a more
# complex set of operations. To capture multiple groups. We also use the
# longer flags, for readability.
#
# The capture groups on this one are complex. This amplicon is designed with
# first five bases being sample index, then it's fixed sequence, then it's
# a strain barcode, then fixed sequence, then 6 bases of UMI alternating with
# fixed sequence. Yep. Kinda complex.

input_fastq=$(cat <<EOF
@NB501157:100:H5J5LBGX2:1:11101:10000:10043 1:N:0:
TTCACGTCCACGAGGTCTCTTCAGTCGTAGCAGTTCGCGTACGCTACAGGTCGACGGTAAGAGAGGGATGTGATC
+
AAAAAEEEA/<EAEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEE
@NB501157:100:H5J5LBGX2:1:11101:10000:19701 1:N:0:
CTACTGTCCACGAGGTCTCTGATGCACTGCGTTCCATGTTCGTACGCTGCAGGTCGACGGAAGGAGCGCGATGTG
+
AAAAAEEEE/AEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
EOF
)

echo "${input_fastq}" | 
    itermae \
        --operation "input > (?P<sample>[ATCG]{5})(?P<fixed1>GTCCACGAGGTC){e<=1}(?P<rest>TCT.*){e<=1}" \
        --operation "rest > (?P<tag>TCT){e<=1}(?P<strain>[ATCG]{10,26})CGTACGC" \
        --operation "rest > (?P<fixed2>CGTACGCTGCAGGTC)(?<UMItail>GAC[ATCG]G[ATCG]A[ATCG]G[ATCG]G[ATCG]G[ATCG]GAT){s<=2}" \
        --operation "UMItail > (GAC(?P<umi1>[ATCG])G(?<umi2>[ATCG])A(?<umi3>[ATCG])G(?<umi4>[ATCG])G(?<umi5>[ATCG])G(?<umi6>[ATCG])G){e<=2}" \
        --output-id "input.id+'_umi='+umi1.seq+umi2.seq+umi3.seq+umi4.seq+umi5.seq+umi6.seq+'_sample='+sample.seq" \
        --output-seq "strain" \
        -v

# This should output one SAM-format entry that matches and can form the output
# appropriately.


In [ ]:
# Demo 01
# 
# This is just a real simple example to start.
# 
# For maximum transparency, in these demos I contrive a fake FASTQ file as
# a string, delimited by the usage of '<<EOF' and 'EOF' as seen below, and feed
# that either directly into the program (as here) or into a temporary file
# (for later demos).

input_fastq=$(cat <<EOF
@NB501157:100:H5J5LBGX2:1:11101:10000:10043 1:N:0:
TTCACGTCCTCGAGGTCTCTTCAGTCGTAGCAGTTCGATGCGTACGCTACAGGTCGACGGTAAGAGAGGGATGTG
+
AAAAAEEEA/<EAEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEE
@NB501157:100:H5J5LBGX2:1:11101:10000:10138 1:N:0:
GCTTCGTCCTCGAGGTCTCTTGGGCAGACACAACGCTACACGTACGCTGCAGGTCGAGGGCACGCGAGAGATGTG
+
AAAAAEEEE/AEAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
EOF
)

echo "${input_fastq}" | 
    ./itermae.py -o "input > (?P<sample>[ATCGN]{5})(?P<after>[ATCGN]{10})" \
        -oid "input.id+'_'+sample.seq" -oseq "after"

# This should output two SAM-format entries, putting the first five bases in the
# read ID and saving the sequence as the next 10 bases after those.

# The above command could be equivalently written with the long-form of the
# flags as --output-id and --output-seq

#		-o "Sample:  input   > (?P<sample>[ATCG]{5})(?P<fixed1>GTCCACGAGGTC){e<=1}(?P<rest>TCT.*){e<=1}" \
#		-o "Strain:  rest	 > (?P<tag>TCT){e<=1}(?P<strain>[ATCG]{10,26})CGTACGCTGCAGGTCGAC" \
#		-o "UMITail: rest    > (?P<fixed2>CGTACGCTGCAGGTC)(?<UMItail>GAC[ATCG]G[ATCG]A[ATCG]G[ATCG]G[ATCG]G[ATCG]GAT){s<=2}" \
#		-o "UMI:     UMItail > (GAC(?P<umi1>[ATCG])G(?<umi2>[ATCG])A(?<umi3>[ATCG])G(?<umi4>[ATCG])G(?<umi5>[ATCG])G(?<umi6>[ATCG])G){e<=2}" \
#		--output-seq "sample+spacer+strain" \
#		--output-id "input.id+'_umi='+umi1.seq+umi2.seq+umi3.seq+ \
#			umi4.seq+umi5.seq+umi6.seq+'_sample='+sample.seq" \
#		--filter "sample_length == 5 and rest_start >= 16" \
#		--output-id "input.id+'_umi='+umi1.seq+umi2.seq+umi3.seq+ \
#			umi4.seq+umi5.seq+umi6.seq+'_sample='+sample.seq" \
#		--filter "sample_length == 5 and rest_start >= 16" \
#		--job-polling-delay 0.1 \
#		--verbose #-m 


In [ ]:
# Demo XX
# 
# Filters
# Filters are evaluated as straight plain python, but where each group is a 
# variable available, and these each have the attributes of 'start', 'end', and
# 'length'.
#
# Here we use a reduced set of operations to keep it sorta simpler.

input_fastq=$(cat <<EOF
@NB501157:100:H5J5LBGX2:1:11101:10000:10043 1:N:0:
TTCACGTCCACGAGGTCTCTTCAGTCGTAGCAGTTCGCGTACGCTACAGGTCGACGGTAAGAGAGGGATGTG
+
AAAAAEEEA/<EAEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEE
@NB501157:100:H5J5LBGX2:1:11101:10000:19701 1:N:0:
CTACTGTCCACGAGGTCTCTGATGCACTGCGTTCCATGTTCGTACGCTGCAGGTCGACGGAAGGAGCGCGATGTG
+
AAAAAEEEE/AEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
EOF
)

echo "${input_fastq}" | 
    itermae \
        --operation "input > (?P<sample>[ATCG]{5})(?P<fixed1>GTCCACGAGGTC){e<=1}(?P<rest>TCT.*){e<=1}" \
        --operation "rest > (?P<tag>TCT){e<=1}(?P<strain>[ATCG]{10,26})(?P<tail>CGTACGC)" \
        --output-id "input.id+'_sample='+sample.seq" \
        --output-seq "strain" \
		--filter "sample.length == 5 and strain.length == 20" 

# Note that the first one is filtered out, not because the 'sample' barcode is 
# not 5 bases at the start, but 'strain' barcode is not exactly 20 bases. It is
# still captured because I specified that 'strain' could capture between 10 and
# 26 bases, but it's not output because the filter statement is not true.


